In [2]:
import pandas as pd
import altair as alt
import os 
from vega_datasets import data

In [3]:
cuisine_mapping = pd.read_csv('data/Cuisinne Mapping.csv')

In [4]:
# List files in data/Taste/
files = os.listdir('data/Taste/')
# Read in all files with pandas
# Filename as a column
dfs = []
for file in files:
    dfs.append(pd.read_csv('data/Taste/' + file))
    
# Concatenate all files
taste = pd.concat(dfs)

In [5]:
# Merge with cuisine mapping
mapping = taste.merge(cuisine_mapping, on='cuisine')
# Filter out those with blank "ISO 31661 Numeric"
mapping_filtered = mapping[mapping['ISO 31661 Numeric'].notnull()]
mapping_filtered.loc[:,'id'] = mapping_filtered['ISO 31661 Numeric'].astype(int).astype(str)

/tmp/ipykernel_2302/816558526.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mapping_filtered.loc[:,'id'] = mapping_filtered['ISO 31661 Numeric'].astype(int).astype(str)


In [6]:
mapping_filtered.head()

,sweetness,saltiness,sourness,bitterness,savoriness,fattiness,spiciness,id,title,cuisine,Country,ISO 31661 Numeric,Notes
0,39.16,100.00,19.60,36.61,52.40,69.90,4800000.0,408,Korean Ground Beef,Korean,South Korea,408.0,NaN
1,39.42,100.00,13.58,23.97,8.43,11.33,0.0,408,Sous Vide Beef Bulgogi Bowls,Korean,South Korea,408.0,NaN
2,14.48,100.00,0.00,0.00,0.00,7.12,0.0,408,Pressure Cooker Korean Beef Sliders,Korean,South Korea,408.0,NaN
3,36.92,100.00,25.93,35.64,47.65,54.12,0.0,408,Dongchimi (Korean Radish Water Kimchi),Korean,South Korea,408.0,NaN
4,100.00,50.93,23.35,64.56,8.63,39.11,0.0,408,Sparkling White Kimchi,Korean,South Korea,408.0,NaN


In [7]:
source = alt.topo_feature(data.world_110m.url, 'countries')
sphere = alt.sphere()
graticule = alt.graticule()

# flavors = mapping_filtered.columns[0:7]
# input_dropdown = alt.binding_select(options=flavors, name='Flavor')
# selection = alt.selection_single(fields=['Origin'], bind=input_dropdown)

flavor = 'sweetness'

# Layering and configuring the components
alt.Chart(source).mark_geoshape(
    stroke='black'
    ).encode(
        color= flavor + ':Q'
    ).transform_lookup(
    lookup='id',
    from_=alt.LookupData(mapping_filtered, 'id', [flavor])
).project(
    'naturalEarth1'
).properties(width=600, height=400).configure_view(stroke=None)

MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000). For information on how to plot larger datasets in Altair, see the documentation

In [8]:
# Same as above, but with a filter for the flavor

# Keep first 8 columns of mapping_filtered
mapping_filtered_flavor = mapping_filtered[mapping_filtered.columns[0:8]]


mapping_filtered_flavor_pivoted = pd.melt(mapping_filtered_flavor, id_vars='id', value_vars=mapping_filtered_flavor.columns[1:8])
# Melt with 'id' as key

In [9]:
# Take average of value by id, variabl
averaged = mapping_filtered_flavor_pivoted.groupby(['id','variable']).mean()
# Reset hierarchical index
averaged = averaged.reset_index()

In [10]:
averaged

,id,variable,value
0,156,bitterness,3.345268e+01
1,156,fattiness,6.659982e+01
2,156,saltiness,8.688942e+01
3,156,savoriness,5.350402e+01
4,156,sourness,2.619758e+01
...,...,...,...
73,840,fattiness,6.700252e+01
74,840,saltiness,7.291652e+01
75,840,savoriness,4.397934e+01
76,840,sourness,4.188214e+01


In [14]:
## NOT WORKING - needs debugged (only showing spiciness for some reason)
input_dropdown = alt.binding_select(options=list(mapping_filtered_flavor.columns[0:7]), name='Flavor')
selection = alt.selection_single(fields=['variable'], bind=input_dropdown)

# Create same map as above but with a filter for the flavor
# Layering and configuring the components
chart = alt.Chart(source).mark_geoshape(
    ).encode(
        color= 'value:Q',
        # opacity = 'variable:N'
    ).transform_lookup(
    lookup='id',
    from_=alt.LookupData(averaged, 'id', ['value','variable'])
).project(
    'equirectangular'
).properties(width=600, height=400
).add_selection(
    selection
).transform_filter(selection).facet(
    facet='variable:N',
    columns=2
)
chart

alt.FacetChart(...)

In [27]:
all_flavors = list(mapping_filtered_flavor.columns[0:7])
all_flavors

['sweetness',
 'saltiness',
 'sourness',
 'bitterness',
 'savoriness',
 'fattiness',
 'spiciness']

In [50]:
charts = []
for selected_flavor in all_flavors:
    # filter source where variable == selected_flavor
    selected_averaged = averaged[averaged['variable'] == selected_flavor]

    chart = alt.Chart(source).mark_geoshape(
        ).encode(
            color= 'value:Q',
            # opacity = 'variable:N'
        ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(selected_averaged, 'id', ['value','variable'])
    ).project(
        'equirectangular'
    ).properties(width=600, height=400, title= selected_flavor
    )
    charts.append(chart)
# Display a grid of charts
alt.vconcat((charts[1] | charts[2]),
(charts[3]| charts[4]),
(charts[5]| charts[6]),
).resolve_scale(
    color='independent'
)

AttributeError: 'Chart' object has no attribute 'tooltip'

In [51]:
chart = alt.vconcat((charts[1] | charts[2]).resolve_scale(color='independent'),
(charts[3]| charts[4]).resolve_scale(color='independent'),
(charts[5]| charts[6]).resolve_scale(color='independent'),
).resolve_scale(
    color='independent'
)
chart

IndexError: list index out of range

In [49]:
print(chart.to_json(indent=2))

{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.17.0.json",
  "config": {
    "view": {
      "continuousHeight": 300,
      "continuousWidth": 400
    }
  },
  "data": {
    "format": {
      "feature": "countries",
      "type": "topojson"
    },
    "url": "https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/world-110m.json"
  },
  "datasets": {
    "data-2b4aa875a2ec0b08a2c41982e31425f7": [
      {
        "id": "156",
        "value": 804556.0192999999,
        "variable": "spiciness"
      },
      {
        "id": "250",
        "value": 155906.4766,
        "variable": "spiciness"
      },
      {
        "id": "276",
        "value": 200386.26668,
        "variable": "spiciness"
      },
      {
        "id": "356",
        "value": 1105402.59668,
        "variable": "spiciness"
      },
      {
        "id": "372",
        "value": 177171.12253999998,
        "variable": "spiciness"
      },
      {
        "id": "380",
        "value": 80279.51329999999,
      